# Detecting spam with Perceptrons

One of the first and successful applications of AI in the field of cybersecurity is Spam Detection, and one of the most famous is **SpamAssassin**

The strategies that can be applied are different, but the most common and simpler one uses **Neural Networks (NNs)**
in the most basic way wich is **Perceptrons**

## NNs at the basics - **the Perceptron**
The Perceptron is one the first successful implementations of neuron in the field of **AI**.

Just like a neuron of human brain is characterized by a layered structure

![neuron of a human brain](./resources/HumanNeuron.jpg)

The artificial rapresentation of the neuron implemented thourgh the Perceptron model is structured in a such way to associate a given output value to one or more levels of input data:

![AI neuron](./resources/AiNeuron.jpg)

The mechanism that transform the input data into an output value is implemented by making use of appropriate weighing of the the values of an input, wich are summarized to an activate function wich, when exceeding a certain threshold,produces a value of output that is forwarded to the remaining components of the neural network.

## We have to find the right weight!

The difference between the stastical models and AI algorithms is that the algorithms implement an optimization strategy based on iteration, for each iteration, the algo tries to adjust the estimate of the values, giving to them a bigger or lower weight depending on the cost that function that we have to minimize. The scope of the algo is to find an optimal weight to estimate values in order to obtain rielable future predictions on unknown future data.

## How a Spam filter works
Imagine that we are going to separete our emails by categorizing them based on the presence ro the absence of certain keywords that occurs in our emails witha certain frequency. To do this we can list on table all the incoming emails and after running the AI function giving them a tag to classify them by two paramenters **HAM** or **SPAM**.

As we said we are going to count the occurence of certain keywords within the text of the email and we will assign a score to each individual message so we can classify all the incoming emails. 

We will identify a threshold value that allows us to separate smap messages. If the score exceeds the threshold value, the mail will be classified as SPAM if not as HAM.
The threshold  value will be constantly updated to take into account the new series of spam emails that we will receive in future.

## Example of Spam Filter
First of all let's classify emails based on sospicious keywork. For simplicity let's consider the most rapresentative sospicious keywords, buy and sex.
So we are going to create a table where for each email we identifie the occurence of each individual keywords, and after that we indicate if the email is Spam or Ham:



| Email|      Buy|      Sex|     Spam or Ham?|
|:-------------|:-----------|:------|:------|
|1|1|0|H|
|2|0|1|H|
|3|0|0|H|
|4|1|1|S|


At this point, we will assign a score to every single email message:
The score will be calculated by a function that takes into account the number of accourences of sospicious keywords contained within text.

A possible scoring function could be the sum of the occureces of our two keywords, represented in this case by the ***B*** variable instead of the world buy, and the ***S*** variable instead of the word sex.

So the function will be:
$$ y = B + S; $$
We can also set different weight to rapresent variables of the respective keywords, based on the fact that, for example, the keyword sex contained within the message is indicative of a greater probability of spam than the word buy.

It is clear that if both words are present in the text of the email, the probability of it being spam increases. Therefore, we will attribute a lower weight of 2 to the ***B*** variable and a greater weight of 3 to the ***S*** variable.

Our scoring function, corrected with the relative weights assigned to the variables/keyword, therefore becomes the following:

$$ y = 2B + 3S; $$
Now let's try to reclassify our emails, calculating the relative scores with our scoring function:






|Email|Buy|Sex|2B + 3S|Spam or Ham?|
|:-------------|:-----------|:------|:------|:------|
|1|1|0|2|H|
|2|0|1|3|H|
|3|0|0|0|H|
|4|1|1|5|S|

At this point, we must try to identify a threshold value that effectively separates spam from ham. Indeed, a threshold value between 4 and 5 allows us to properly separate the spam from the ham.

Let's translate all of this in mathematical formulas, to use it in our algorithms..

So our function to determine the score is: 

$$ y = 2B + 3S $$

This identify a straight line in the Cartesian plane, so the classifier used by our spam filter is a **linear classifier**.
If we substitute oru ***B*** and ***S*** value with a matrix ***Xi*** cointaing them, and after introducing a vector ***wi*** we can say:

$$ y = \sum w_{i}x_{i} = w_{1}x_{1} + w_{2}x_{2} + \dot +w_{n}x_{n} $$

with index ***i*** going from 1 to ***n***.

This way we have genralized out linead classifier to an unspecified number of variables, ***n***, rather than limiting ourselves to just 2 keywords.

In fact, our function translates into a sum of products that can easily be representend as a product of matrices and vector:
$$ y = wTx $$
***wT*** stands for the transported weights carrier, necessary calculating the product of the matrices and vectors.

To rapresent the threshold value in formal terms we are gonna say:

$$ if, wx> \theta \to f(y) = +1 $$
$$ if, wx< \theta \to f(y) = -1 $$

That translates to:

$$ w_{1}x_{1} + w_{2}x_{2} + ... +w_{n}x_{n} \geq \theta \to y = + 1 $$
$$ w_{1}x_{1} + w_{2}x_{2} + ... +w_{n}x_{n} < \theta \to y = - 1 $$

